## Base Inference

In [1]:
from transformers import Pix2StructForConditionalGeneration, Pix2StructProcessor
from PIL import Image

model = Pix2StructForConditionalGeneration.from_pretrained("google/pix2struct-infographics-vqa-base").to("cuda")
processor = Pix2StructProcessor.from_pretrained("google/pix2struct-infographics-vqa-base")


In [2]:
image = Image.open("/home/jjh/level3-cv-productserving-cv-10/data/images/10065.jpeg")
question = "Which market crash had the lowest impact on the S&P 500, Dot-com crash, Coronavirus crash, or Great recession ?"
inputs = processor(images=image, text=question, return_tensors="pt").to("cuda")

In [3]:
inputs["flattened_patches"].shape

torch.Size([1, 2048, 770])

In [4]:
ins = processor(images = image,text=question,return_tensors='pt').to('cuda')

In [5]:
predictions = model.generate(**inputs)
pred = processor.decode(predictions[0], skip_special_tokens=True)
print(pred)

Great recession


/home/jjh/anaconda3/lib/python3.11/site-packages/transformers/generation/utils.py:1254: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


## train

In [1]:
import os
import torch
from torch.utils.data import Dataset
import json
from PIL import Image

/home/lym/miniconda3/envs/layoutv3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import Pix2StructForConditionalGeneration, AutoProcessor

model = Pix2StructForConditionalGeneration.from_pretrained("google/pix2struct-infographics-vqa-base").to("cuda")
auto_processor = AutoProcessor.from_pretrained("google/pix2struct-infographics-vqa-base")

In [3]:
img_dir = './data/images/'
train_dir = './data/qas/infographicsVQA_train_v1.0.json'
val_dir = './data/qas/infographicsVQA_val_v1.0_withQT.json'
test_dir = './data/qas/infographicsVQA_test_v1.0.json'

with open(train_dir, 'r', encoding='utf-8') as train_data:
    train_json_data = json.load(train_data)
with open(train_dir, 'r', encoding='utf-8') as val_data:
    val_json_data = json.load(val_data)
with open(test_dir, 'r', encoding='utf-8') as test_data:
    test_json_data = json.load(test_data)        

In [4]:
class Pix2StructDataset(Dataset):
    def __init__(self, image_dir, json_data, processor, train):
        self.img_dir = image_dir
        self.json_data = json_data['data'][:20]
        self.processor = processor
        self.file_list = os.listdir(image_dir)
        self.train = train
        
    def __len__(self): 
        return len(self.json_data)
        
    def __getitem__(self, index): 
        data = self.json_data[index]
        image_name = data["image_local_name"]
        img = Image.open(os.path.join(self.img_dir, image_name))
        q = data["question"]
        inputs = self.processor(images=img, text=q, return_tensors="pt")
        if self.train:
            a = data["answers"][0]
            label = self.processor.tokenizer(text=a, padding="max_length", return_tensors="pt", add_special_tokens=True, max_length=45).input_ids
            inputs["labels"] = label
            return inputs
        return inputs
  
    
    

In [5]:
def collator(batch):
  new_batch = {"flattened_patches":[], "attention_mask":[], "labels":[]}
  
  for item in batch:
    new_batch["flattened_patches"].append(item["flattened_patches"][0])
    new_batch["attention_mask"].append(item["attention_mask"][0])
    new_batch["labels"].append(item["labels"][0])
  new_batch["flattened_patches"] = torch.stack(new_batch["flattened_patches"])
  new_batch["attention_mask"] = torch.stack(new_batch["attention_mask"])
  new_batch["labels"] = torch.stack(new_batch["labels"])

  return new_batch

In [6]:
train_dataset = Pix2StructDataset(image_dir=img_dir, json_data=train_json_data, processor=auto_processor, train=True)
val_dataset = Pix2StructDataset(image_dir=img_dir, json_data=val_json_data, processor=auto_processor, train=True)
test_dataset = Pix2StructDataset(image_dir=img_dir, json_data=test_json_data, processor=auto_processor, train=False)


In [7]:
def load_json(json_dir):
    with open(json_dir) as f:
        json_data = json.load(f)
    return json_data

In [8]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    report_to="wandb",
    run_name="pix2struct_jjh",
    output_dir="pix2struct_1",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    fp16=True
)

In [10]:
from datasets import load_metric
import numpy as np

accuracy_metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred.predictions, eval_pred.label_ids
    predictions = np.argmax(predictions, axis=1)
    # metrics from the datasets library have a `compute` method
    return accuracy_metric.compute(predictions=predictions, references=labels)

/tmp/ipykernel_600966/2834543161.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  accuracy_metric = load_metric("accuracy")
/home/lym/miniconda3/envs/layoutv3/lib/python3.11/site-packages/datasets/load.py:756: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [11]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=collator
    compute_metrics=compute_metrics
)

trainer.train()

/home/lym/miniconda3/envs/layoutv3/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None)
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: edwin-minlee (level2-cv-10-detection). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,No log,31.663639


RuntimeError: [enforce fail at inline_container.cc:325] . unexpected pos 398029376 vs 398029268

--- Logging error ---
Traceback (most recent call last):
  File "/home/lym/miniconda3/envs/layoutv3/lib/python3.11/logging/__init__.py", line 1114, in emit
    self.flush()
  File "/home/lym/miniconda3/envs/layoutv3/lib/python3.11/logging/__init__.py", line 1094, in flush
    self.stream.flush()
OSError: [Errno 28] No space left on device
Call stack:
  File "/home/lym/miniconda3/envs/layoutv3/lib/python3.11/threading.py", line 1002, in _bootstrap
    self._bootstrap_inner()
  File "/home/lym/miniconda3/envs/layoutv3/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/home/lym/miniconda3/envs/layoutv3/lib/python3.11/site-packages/wandb/sdk/internal/internal_util.py", line 49, in run
    self._run()
  File "/home/lym/miniconda3/envs/layoutv3/lib/python3.11/site-packages/wandb/sdk/internal/internal_util.py", line 100, in _run
    self._process(record)
  File "/home/lym/miniconda3/envs/layoutv3/lib/python3.11/site-packages/wandb/sdk/internal/internal.py", l